# The Turtle's Trail

#### The design and implementation of Jupyturtle (so far)

As I reviewed Allen Downey's *Think Python Third Edition*,
I thought he could use a simpler and more adaptable
module to make turtle graphics, like Python's `turtle` module,
but running inside a Jupyter Notebook instead of desktop app.

I've been a big fan of turtle graphics since I taught programming
for kids using Logo in the 1980's.
I also wrote a Logo world (environment) for adults to practice
programming. I called it Logol. It was inspired by the book
*Karel the Robot* which taught algorithmic thinking through
a software environment where learners programmed a robot to
move and perform tasks on a grid of streets and avenues.
You can think of it as "turtle on rails".

The core idea of Jupyturtle I stole from Tolga Atam's
[ColabTurtle](https://github.com/tolgaatam/ColabTurtle):
the simplest way to draw vector graphics on a Jupyter
Notebook is to generate SVG.

## Hello Turtle World

Here is the turtle's "Hello World!" with its output:

In [1]:
from jupyturtle import *

fd(100)

The green elements are the turtle's head and body. It starts facing right by default.

The purple line is the turtle's trail.
The turtle draws a line when it moves, but there ways to move without drawing.

`fd(100)` means "move forward 100 units".
You can also write `forward` but after a while the two letter names become natural.

`fd` is one of a few special functions known as *turtle commands*.
Guess what these other commands do: `lt(90)`, `bk(50)`, `rt(45)`.

Let's see:

In [2]:
lt(90)
bk(50)
rt(45)

Note that the commands were sent to the latest turtle rendered in the notebook.

If you want to start a new drawing, use the `%%turtle` magic command at the top of the cell:

In [3]:
%%turtle

fd(60)
lt(120)
fd(60)
lt(120)
fd(60)


We can (but rarely do) inspect the generated SVG:

In [4]:
print(get_turtle().get_SVG())

<svg width="300" height="150" style="fill:none; stroke-linecap:round;">
    <rect width="100%" height="100%" fill="#F3F3F7" />


<path stroke="#663399" stroke-width="2" d="M 150,75 210,75 180,23 150,75" />'

<g transform="rotate(30.0,150.0,75.0) translate(150.0, 75.0)">
    <circle stroke="#63A375" stroke-width="2" fill="transparent" r="5.5" cx="0" cy="0"/>
    <polygon points="0,12 2,9 -2,9" style="fill:#63A375;stroke:#63A375;stroke-width:2"/>
</g>

</svg>


The `<path>` element draws connected line segments from coordinate pairs in the `d=` attribute.

The `<g>` element groups, rotates, and translates a `circle` and a `polygon` to draw the turtle.

In [5]:
%%turtle fast

def polyspi(side, angle):
    fd(side)
    rt(angle)
    if side < 140:
        polyspi(side + 4, angle)

polyspi(5, 144)

If you use the `fast` option with `%%turtle`, Jupyturtle skips the animation and shows the final drawing.

## Turtle magic

The `%%turtle` magic command creates a new turtle to start a new drawing.
The global function `make_turtle()` does the same.

`%%turtle` must be used at the top of a code cell, after importing `jupyturtle` in a previous cell.

You can pass optional arguments to `%%turtle` like this:

```
%%turtle arg1 arg2 arg3
```

If one of the arguments is the word "fast" (no quotes), animation is turned off.

If two of them are numbers, they set the drawing width and height in that order.

If only one is a number, it sets both width and height so you get a square drawing area.

## Implementation details

Jupyturtle is a single module `jupyturtle.py`, to make it easy to install on Jupyter Notebook kernels running on remote machines. It is also a [Proper Package™ on PyPI](https://pypi.org/project/jupyturtle/).

The module provides imperative and object-oriented APIs. 
The imperative API consists of top-level functions that work as turtle commands like `fd(50)` etc.
The OO API is provided by the `Turtle` class, which has methods such as `my_turtle.forward(50)`.

The imperative API is built automatically at import time on top of the OO API, to avoid duplication.

When you use a turtle command, behind the scenes it calls `get_turtle()`,
which returns the pre-existing "main turtle", or creates a new one if there was no main turtle.
The command then calls its corresponding method on the main turtle.
So `fd(90)` is the same as `get_turtle().forward(90)`.

### Metaprogramming to build commands from methods

The `@command` and `@command_alias` decorators in `jupyturtle.py` register
methods of the `Turtle` class that we want to work as top-level module functions.

`@command` is very simple:

```python
# mapping of method names to global aliases
_commands = {}


# decorators to build procedural API with turtle commands
def command(method):
    """Register method for use as a top level function in procedural API."""
    _commands[method.__name__] = []  # no alias
    return method
```

That's a typical registration decorator:
it collects information such as `__name__` from the decorated function or method,
then returns it unchanged.
There's no inner function to wrap and replace the decorated one.

Here is what the `_commands` mapping holds after `jupyturtle` is imported:

In [6]:
from jupyturtle import jupyturtle  # get module inside package
jupyturtle._commands

{'draw': [],
 'hide': [],
 'show': [],
 'move_to': ['moveto', 'mv'],
 'forward': ['fd'],
 'leap': ['lp'],
 'back': ['bk'],
 'jump_to': ['jumpto', 'jp'],
 'left': ['lt'],
 'right': ['rt'],
 'pen_up': ['penup', 'pu'],
 'pen_down': ['pendown', 'pd'],
 'toggle_pen': [],
 'repeat': []}

The `@command_alias` accepts aliases (alternative names) for the decorated function.
As you can see above, `pen_up` has two aliases: `penup` and `pu`.

This how `pen_up` is decorated in the `Turtle` class:

```python
class Turtle:

    # ...many methods omitted here...
    
    @command_alias('penup', 'pu')
    def pen_up(self):
        """Lift the pen, so turtle stops drawing."""
        self.active_pen = False
```

The code for `@command_alias` is a little more complicated:

```python
def command_alias(*names):
    """Same as @command, but assigning aliases to the top level function."""

    def decorator(method):
        _commands[method.__name__] = list(names)
        return method

    return decorator
```

The outer function `command_alias` exists only to collect the arguments.
It then defines and returns an inner function which is the actual decorator:
a fuction that gets the method to be decorated.

When `jupyturtle` is imported, `@command` and `@command_alias` fill the `_commands` mapping.

Then `_install_commands` is called at the top level of the module:
it iterates over `_commands` and calls `_make_command` and `_install_command` on each registered method.

```python
def _install_commands():
    for name, aliases in _commands.items():
        new_command = _make_command(name)
        _install_command(name, new_command)
        for alias in aliases:
            _install_command(alias, new_command)


_install_commands()
```

`_make_command` gets the named unbound method from the `Turtle` class and creates a new function
which gets the main turtle instance with `get_turtle()`, then calls the method on that instance.
It also replaces the `__name__` and `__doc__` of the new function with those of the named method.

```python
def _make_command(name):
    method = getattr(Turtle, name)  # get unbound method

    def command(*args, **kwargs):
        turtle = get_turtle()
        method(turtle, *args, **kwargs)

    command.__name__ = name
    command.__doc__ = method.__doc__
    return command
```

The `_install_command` function to insert each new command and its aliases in the module's global namespace
and also in the `__all__` attribute which controls the names that are exported when you write `from jupyturtle import *`.

```python
def _install_command(name, function):
    if name in globals():
        raise ValueError(f'duplicate turtle command name: {name}')
    globals()[name] = function
    __all__.append(name)
```
